## Notebook to demo ReAct geospatial logic with Ollama.

Following is based on [this](https://python.langchain.com/docs/modules/agents/agent_types/react) reference.

In [1]:
import json
import math
import re
from typing import Dict, Optional, Tuple, Type, Union

from langchain import hub
from langchain.agents import create_react_agent, load_tools
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, tool
from langchain_core.tools import Tool
from langchain_experimental.tools.python.tool import PythonREPLTool

### Create LLM backed by ollama.

Set `temperature` to zero to return more precise responses.

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="mistral:latest",
    temperature=0.0,
)

## Get a prompt.

In [3]:
prompt = hub.pull("hwchase17/react")

In [4]:
# print(prompt.template)

### Define a dummy locator.

In [5]:
class Locator:
    def __init__(
            self,
            locations: Dict[str, Tuple[float, float]],
    ) -> None:
        """Initialize with location table"""
        self.locations = locations

    def locate(
            self,
            location: str,
    ) -> str:
        """Locate input query.

        :param location: The location to locate.
        :return: json string containing lon/lat.
        """
        lon, lat = self.locations.get(location.lower(), (None, None))
        return json.dumps({"lon": lon, "lat": lat})

### Define the geocode tool input.

In [6]:
class GeocodeInput(BaseModel):
    query: str = Field(description="An address, place or country to search and geocode")

### Define the geocoder tool.

In [7]:
class GeocodeTool(BaseTool):
    name = "geocode_tool"
    description = """
    Use this tool to get the latitude and longitude coordinate of an address, a location, a place or a country.
    """
    args_schema: Type[BaseModel] = GeocodeInput
    locator: Locator = Field(exclude=True)

    def _run(
            self,
            query: str,
            run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Delegate to locator."""
        return self.locator.locate(query)

    async def _arun(
            self,
            query: str,
            run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Run async locator."""
        return self.locator.locate(query)

### Create a geocoder tool instance using a locator instance with dummy locations.

In [8]:
locations = {
    "beirut, lebanon": (33.8938, 35.5018),
    "beirut": (33.8938, 35.5018),
    "boston": (42.3601, -71.0589),
    "singapore": (1.3521, 103.8198),
}

geocode_tool = GeocodeTool(locator=Locator(locations))

In [9]:
geocode_tool.run("beirut")

'{"lon": 33.8938, "lat": 35.5018}'

### Create extent tool.

Just another way to define a tool.

In [10]:
class ExtentInput(BaseModel):
    doc: str = Field(description="A document with lon and lat keys.")


def calculate_extent(doc: str) -> str:
    """Calculate an extent from lon/lat values in the doc."""
    pattern = r"\d+\.\d+"
    tokens = re.findall(pattern, doc)
    if len(tokens) == 2:
        lon = float(tokens[0])
        lat = float(tokens[1])
        xmin = lon - 10.0
        xmax = lon + 10.0
        ymin = lat - 10.0
        ymax = lat + 10.0
        return json.dumps({"xmin": xmin, "xmax": xmax, "ymin": ymin, "ymax": ymax})
    else:
        return json.dumps({"xmin": 0.0, "xmax": 0.0, "ymin": 0.0, "ymax": 0.0})


async def acalculate_extent(doc: str) -> str:
    # Async version of the calculate_extent function
    return calculate_extent(doc)


extent_tool = Tool(
    name="extent_tool",
    description="""
    A tool that calculates an extent around a given latitude and logitude values.
    """,
    args_schema=ExtentInput,
    func=calculate_extent,
    coroutine=acalculate_extent,
    # return_direct=True,
)

### Create tools using decorators.

In [11]:
@tool
def calculate_circumference(radius: Union[str, int, float]) -> float:
    """Use this tool when you need to calculate a circumference  of a circle using the radius."""

    # Regular expression pattern to match a number (integer or decimal)
    pattern = r"\d+(\.\d+)?"

    # Searching for the pattern in the input string
    match = re.search(pattern, radius)
    return float(match.group()) * 2.0 * math.pi if match else float("NaN")

In [12]:
@tool
def calculate_area(radius: Union[str, int, float]) -> float:
    """Use this tool when you need to calculate an area of a circle using the radius."""

    # Regular expression pattern to match a number (integer or decimal)
    pattern = r"\d+(\.\d+)?"

    # Searching for the pattern in the input string
    match = re.search(pattern, radius)
    rad = float(match.group())
    return rad * rad * math.pi if match else float("NaN")

### Define the tools.

In [13]:
repl = PythonREPLTool()  # A tool for running python code in a REPL.

In [14]:
tools1 = [calculate_circumference, calculate_area, geocode_tool, extent_tool, repl]

### Load tool to perform "math".

In [15]:
tools2 = load_tools(["llm-math"], llm=llm)

In [16]:
tools = tools1 + tools2

### Create ReAct Agent and Excutor.

In [17]:
agent = create_react_agent(llm, tools, prompt)

In [18]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=10,
    early_stopping_method="generate",
    max_tokens=512,
)

### Invoke ReAct chain.

In [19]:
resp = agent_executor.invoke(
    {
        "input": "Calculate and return the extent for Beirut",
    }
)
resp["output"]



> Entering new AgentExecutor chain...
 To calculate the extent for Beirut, I first need to find its latitude and longitude coordinates using the geocode_tool. Then, I can use the extent_tool to calculate the extent based on those coordinates.

Action: geocode_tool
Action Input: Beirut, Lebanon{"lon": 33.8938, "lat": 35.5018} Now that I have the latitude and longitude coordinates for Beirut, I can use the extent_tool to calculate the extent.

Action: extent_tool
Action Input: 35.5018, 33.8938{"xmin": 25.501800000000003, "xmax": 45.5018, "ymin": 23.8938, "ymax": 43.8938} I now know the final answer.

Final Answer: The extent for Beirut is {"xmin": 25.5018, "xmax": 45.5018, "ymin": 23.8938, "ymax": 43.8938}

> Finished chain.


'The extent for Beirut is {"xmin": 25.5018, "xmax": 45.5018, "ymin": 23.8938, "ymax": 43.8938}'

### Let's make the agent perform math.

In [20]:
resp = agent_executor.invoke(
    {
        "input": """
        What is the square root of the area of a circle with a 100 meter radius?
        """
    }
)
resp["output"]



> Entering new AgentExecutor chain...
 To find the square root of the area of a circle with a given radius, I first need to calculate the area using the formula Area = πr², then find the square root of that value.

Action: calculate_area
Action Input: 10031415.926535897932

Python REPL can execute arbitrary code. Use with caution.


 Now I have the area of the circle, which is approximately 31,416 square meters. To find the square root of this value, I will use Python's built-in `math.sqrt()` function.

Action: Python_REPL
Action Input: import math; print(math.sqrt(31415.926535897932))177.2453850905516
 I now know the final answer. The square root of the area of a circle with a 100 meter radius is approximately 177.25 meters.Invalid Format: Missing 'Action:' after 'Thought: I apologize for the mistake in formatting. Here's the corrected version:

Final Answer: The square root of the area of a circle with a 100 meter radius is approximately 177.25 meters.

> Finished chain.


'The square root of the area of a circle with a 100 meter radius is approximately 177.25 meters.'